In [1]:
from rich import print

In [2]:
import pandas as pd
from ydata_profiling import ProfileReport

data = pd.read_csv("../../data/structured/carbon.csv")
# profile = ProfileReport(data, title="Carbon Emissions Data Profiling Report", explorative=True)
# profile.to_file('../../data/files/analysis.html')

In [3]:
data.drop(columns=['TID', 'logger'],inplace=True)

In [4]:
def organizar_por_timestamp_context(df, col_timestamp='timestamp', col_context='context'):
    """
    Ordena o DataFrame pelo timestamp crescente e depois pelo context.
    
    Parâmetros:
        df: pandas.DataFrame
        col_timestamp: nome da coluna de timestamp
        col_context: nome da coluna de contexto
    Retorna:
        DataFrame ordenado
    """
    # Garantir que a coluna de timestamp é datetime
    df[col_timestamp] = pd.to_datetime(df[col_timestamp])
    # Ordenar pelos dois campos
    df_ordenado = df.sort_values(by=[col_context,col_timestamp], ascending=[True, True]).reset_index(drop=True)
    return df_ordenado

data = organizar_por_timestamp_context(data)


In [5]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['ano'] = data['timestamp'].dt.year
data['mes'] = data['timestamp'].dt.month
data['dia'] = data['timestamp'].dt.day
data['hora'] = data['timestamp'].dt.hour
data['minuto'] = data['timestamp'].dt.minute
data['segundo'] = data['timestamp'].dt.second
data.drop(columns=['timestamp'], inplace=True)


In [6]:
print(f'Total de ano diferentes: {data["ano"].nunique()}')
print(f'Total de mes diferentes: {data["mes"].nunique()}')
print(f'Total de dia diferentes: {data["dia"].nunique()}')
print(f'Total de hora diferentes: {data["hora"].nunique()}')
print(f'Total de minuto diferentes: {data["minuto"].nunique()}')
print(f'Total de segundo diferentes: {data["segundo"].nunique()}')

Total de ano diferentes: 2

Total de mes diferentes: 3

Total de dia diferentes: 31

Total de hora diferentes: 24

Total de minuto diferentes: 60

Total de segundo diferentes: 60

In [7]:
index = 0
for level in data['level']:
    if level in ['ERROR','WARN']:
        print(f'Index: {index}, Level: {level}')
        break
    index += 1
display(data.iloc[index])

Index: 0, Level: ERROR

level                                                  ERROR
context                             API_LOGGER._OpenService_
message    Error while building message. Error while buil...
ano                                                     2024
mes                                                       12
dia                                                        3
hora                                                      19
minuto                                                    37
segundo                                                   10
Name: 0, dtype: object

In [8]:
# Seleciona mensagens únicas do nível ERROR
mensagens_erro = data['message'][data['level'] == 'ERROR'].unique()

# Salva cada mensagem em uma nova linha de um arquivo .txt
with open("../../data/files/unique_error.txt", "w", encoding="utf-8") as f:
    for mensagem in mensagens_erro:
        f.write(str(mensagem) + "\n")


In [9]:
import pandas as pd

def filtrar_e_salvar_erros(
    df, 
    coluna_msg='message', 
    coluna_nivel='level', 
    termo_excluir='#getNewJsonPayload',
    nivel_erro='ERROR',
    caminho_saida='unique_error.txt'
):
    # Remove linhas com o termo indesejado
    dataM = df[~df[coluna_msg].str.contains(termo_excluir, na=False)].reset_index(drop=True)
    
    # Seleciona mensagens únicas do nível especificado
    mensagens_erro = dataM[coluna_msg][dataM[coluna_nivel] == nivel_erro].unique()
    
    # Salva no arquivo txt
    with open(caminho_saida, "w", encoding="utf-8") as f:
        for mensagem in mensagens_erro:
            f.write(str(mensagem) + "\n")
    
    return dataM.reset_index(drop=True)

# Exemplo de uso:


In [10]:
dataM = filtrar_e_salvar_erros(data, caminho_saida="../../data/files/unique_error1.txt")
dataCNHR = filtrar_e_salvar_erros(dataM,caminho_saida="../../data/files/unique_error2.txt",termo_excluir='Could not handle request')
dataDSAF = filtrar_e_salvar_erros(dataCNHR,caminho_saida="../../data/files/unique_error3.txt",termo_excluir='Deployment of synapse artifact failed')
dataFSNRT = filtrar_e_salvar_erros(dataDSAF,caminho_saida="../../data/files/unique_error4.txt",termo_excluir='Failed to start new registry transaction')
dataKMRKMAIT = filtrar_e_salvar_erros(dataFSNRT,caminho_saida="../../data/files/unique_error5.txt",termo_excluir='Key Manager Resident Key Manager already initialized in tenant')
dataKMRKMAIT = filtrar_e_salvar_erros(dataKMRKMAIT,caminho_saida="../../data/files/unique_error6.txt",termo_excluir='HttpException occurred')

In [11]:
df = dataKMRKMAIT

In [12]:
print(f"Total de mensagens filtradas: {len(df)}")

Total de mensagens filtradas: 867892

In [ ]:
data